In [4]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import sklearn
import keras


In [6]:
# I had Tensor flow backened therefore had to chane it to Theano backend for a simpler approach to this scenario
from keras import backend as K
import os

def set_keras_backend(backend):

    if K.backend() != backend:
        os.environ['KERAS_BACKEND'] = backend
        reload(K)
        assert K.backend() == backend
set_keras_backend("theano")        


Using Theano backend.
/Users/neel/anaconda2/lib/python2.7/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [21]:
df = pd.read_csv('/Users/neel/Downloads/diabetes.csv')

In [22]:
df[:9]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
5,5,116,74,0,0,25.6,0.201,30,0
6,3,78,50,32,88,31.0,0.248,26,1
7,10,115,0,0,0,35.3,0.134,29,0
8,2,197,70,45,543,30.5,0.158,53,1


In [23]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [25]:
# Have some missing data to tackle before I proceed
df[df['Glucose']==0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [30]:
Columns = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']

for col in Columns:
    df[col].replace(0,np.NaN,inplace=True)
df.describe()    

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [31]:
# Need to attain consistency in count of attributes for which i need to drop the missing values
df.dropna(inplace =True)

In [32]:
df.describe()  

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [35]:
# time to convert dataframe to numpy arrays
dataset = df.values
# Split dataset 
X= dataset[:,0:8]
Y =dataset[:,8].astype(int)

In [37]:
# Time to normalize the X part with sklearn standard scaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X)
New_X= scaler.transform(X)

In [39]:
print(New_X)

[[-0.7174265  -1.09104581 -0.37365481 ... -0.710421   -1.03187632
  -0.9682991 ]
 [-1.02921274  0.46631407 -2.45696436 ...  1.42673006  5.11511079
   0.2095853 ]
 [-0.09385402 -1.44794079 -1.65569146 ... -0.29723846 -0.79712575
  -0.47751393]
 ...
 [-0.40564026 -1.12349081 -1.01467313 ... -0.66767798  0.70411863
  -0.87014206]
 [ 2.08864966 -0.70170584  0.42761809 ... -0.02653266 -1.0202837
   3.15429628]
 [ 0.52971846 -0.05280589  0.10710893 ... -0.9811268  -0.80582021
  -0.0848858 ]]


In [53]:
Final_X= pd.DataFrame(New_X)
Y=pd.DataFrame(Y)
print(Final_X)

            0         1         2         3         4         5         6  \
0   -0.717427 -1.091046 -0.373655 -0.585110 -0.522842 -0.710421 -1.031876   
1   -1.029213  0.466314 -2.456964  0.557421  0.100631  1.426730  5.115111   
2   -0.093854 -1.447941 -1.655691  0.271788 -0.573394 -0.297238 -0.797126   
3   -0.405640  2.413014 -0.053146  1.509530  3.260122 -0.368477 -1.057960   
4   -0.717427  2.153454 -0.854419 -0.585110  5.812990 -0.425468 -0.362402   
5    0.529718  1.407219  0.107109 -0.965953  0.159608 -1.038117  0.185349   
6   -1.029213 -0.150141  1.068636  1.699951  0.623000  1.811417  0.081015   
7   -0.717427 -0.636816 -3.258237  0.843053 -0.615520  1.455225 -0.985506   
8   -0.717427 -0.247476 -0.053146  0.081366 -0.505991  0.215678  0.017256   
9   -0.093854  0.109419  1.389146  1.128686  0.665127  0.885318  0.524433   
10   2.400436  0.660984  1.869909  0.366999 -0.084726  0.500631 -0.779737   
11   2.088650  0.076974 -0.053146 -0.299477 -0.345911 -0.282991 -0.921746   

In [41]:
Final_X.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [55]:
from sklearn.model_selection import GridSearchCV,KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.optimizers import Adam

In [45]:
def create_model():
    # Created the Model
    model = Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(4,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    # Compile the model
    adam =Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer ='adam',metrics=['accuracy'])
    return model
model =create_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


In [59]:
seed =6
np.random.seed(seed)
# Defined the model
def create_model():
    # Created the Model
    model = Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(4,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    # Compile the model
    adam =Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer ='adam',metrics=['accuracy'])
    return model
model =create_model()
print(model.summary())

# Create a model

model =KerasClassifier(build_fn=create_model,verbose=0)
#Defining the grid search parameters
batch_size =[10,20,40]
epochs =[10,50,100]
# for grid search parameters cv i have change these to dictionary
param_grid= dict(batch_size=batch_size,epochs=epochs)

# built and fit the grid search
# verbose will print out more things
grid = GridSearchCV(estimator= model, param_grid= param_grid,cv= KFold(random_state=seed),verbose =10)
grid_results=grid.fit(Final_X,Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))








_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_59 (Dense)             (None, 4)                 36        
_________________________________________________________________
dense_60 (Dense)             (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None
Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] epochs=10, batch_size=10 ........................................
[CV] ... epochs=10, batch_size=10, score=0.732824436581, total=   6.3s
[CV] epochs=10, batch_size=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.4s remaining:    0.0s


[CV] ... epochs=10, batch_size=10, score=0.748091597593, total=   6.1s
[CV] epochs=10, batch_size=10 ........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   12.6s remaining:    0.0s


[CV] ... epochs=10, batch_size=10, score=0.799999993581, total=   5.9s
[CV] epochs=50, batch_size=10 ........................................


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  2.8min remaining:    0.0s


[CV] .. epochs=100, batch_size=10, score=0.815384612634, total=  43.0s
[CV] epochs=10, batch_size=20 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  3.5min remaining:    0.0s


[CV] ... epochs=10, batch_size=20, score=0.610687024266, total=   5.6s
[CV] epochs=10, batch_size=20 ........................................
[CV] ... epochs=10, batch_size=20, score=0.748091613973, total=   4.9s
[CV] epochs=10, batch_size=20 ........................................
[CV] ... epochs=10, batch_size=20, score=0.761538468874, total=   5.0s
[CV] epochs=50, batch_size=20 ........................................
[CV] ... epochs=50, batch_size=20, score=0.732824424751, total=  14.9s
[CV] epochs=50, batch_size=20 ........................................
[CV] ... epochs=50, batch_size=20, score=0.770992379607, total=  15.2s
[CV] epochs=50, batch_size=20 ........................................
[CV] ... epochs=50, batch_size=20, score=0.838461545797, total=  16.1s
[CV] epochs=100, batch_size=20 .......................................
[CV] ... epochs=100, batch_size=20, score=0.75572519448, total=  30.7s
[CV] epochs=100, batch_size=20 .......................................
[CV] .

KeyboardInterrupt: 

In [61]:
seed =6
np.random.seed(seed)
# Defined the model
def create_model():
    # Created the Model
    model = Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(4,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    # Compile the model
    adam =Adam(lr=0.01)
    model.compile(loss='binary_crossentropy', optimizer ='adam',metrics=['accuracy'])
    return model
model =create_model()
print(model.summary())

# Create a model

model =KerasClassifier(build_fn=create_model,verbose=0)
#Defining the grid search parameters
batch_size =[10,20,40]
epochs =[10,50,100]
# for grid search parameters cv i have change these to dictionary
param_grid= dict(batch_size=batch_size,epochs=epochs)

# built and fit the grid search
# verbose will print out more things
grid = GridSearchCV(estimator= model, param_grid= param_grid,cv= KFold(random_state=seed),verbose =10)
grid_results=grid.fit(Final_X,Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_226 (Dense)            (None, 8)                 72        
_________________________________________________________________
dense_227 (Dense)            (None, 4)                 36        
_________________________________________________________________
dense_228 (Dense)            (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None
Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] epochs=10, batch_size=10 ........................................
[CV] ... epochs=10, batch_size=10, score=0.732824436581, total=   6.1s
[CV] epochs=10, batch_size=10 ........................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.2s remaining:    0.0s


[CV] ... epochs=10, batch_size=10, score=0.748091597593, total=   5.8s
[CV] epochs=10, batch_size=10 ........................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   12.1s remaining:    0.0s


[CV] ... epochs=10, batch_size=10, score=0.799999993581, total=   6.3s
[CV] epochs=50, batch_size=10 ........................................


...

[CV] .. epochs=100, batch_size=10, score=0.815384612634, total=  45.5s
[CV] epochs=10, batch_size=20 ........................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  3.6min remaining:    0.0s


[CV] ... epochs=10, batch_size=20, score=0.610687024266, total=   5.1s
[CV] epochs=10, batch_size=20 ........................................
[CV] ... epochs=10, batch_size=20, score=0.748091613973, total=   5.0s
[CV] epochs=10, batch_size=20 ........................................
[CV] ... epochs=10, batch_size=20, score=0.761538468874, total=   5.2s
[CV] epochs=50, batch_size=20 ........................................
[CV] ... epochs=50, batch_size=20, score=0.732824424751, total=  16.1s
[CV] epochs=50, batch_size=20 ........................................
[CV] ... epochs=50, batch_size=20, score=0.770992379607, total=  16.0s
[CV] epochs=50, batch_size=20 ........................................
[CV] ... epochs=50, batch_size=20, score=0.838461545797, total=  16.0s
[CV] epochs=100, batch_size=20 .......................................
[CV] ... epochs=100, batch_size=20, score=0.75572519448, total=  28.9s
[CV] epochs=100, batch_size=20 .......................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  8.1min finished


Best: 0.780612250828, using {'epochs': 50, 'batch_size': 20}
0.76020408072 (0.0287186251388) with: {'epochs': 10, 'batch_size': 10}
0.778061220841 (0.0504383094677) with: {'epochs': 50, 'batch_size': 10}
0.762755097175 (0.0372034629249) with: {'epochs': 100, 'batch_size': 10}
0.706632659599 (0.0681946851921) with: {'epochs': 10, 'batch_size': 20}
0.780612250828 (0.0436338811963) with: {'epochs': 50, 'batch_size': 20}
0.778061230116 (0.0217852279412) with: {'epochs': 100, 'batch_size': 20}
0.716836723898 (0.0134015930401) with: {'epochs': 10, 'batch_size': 40}
0.770408165242 (0.0402471085098) with: {'epochs': 50, 'batch_size': 40}
0.767857139968 (0.0337325512793) with: {'epochs': 100, 'batch_size': 40}


In [63]:
# Do a grid search for learning rate and dropout rate
# import necessary packages
from keras.layers import Dropout

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(learn_rate, dropout_rate):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(4, input_dim = 8, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = learn_rate)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1]
dropout_rate = [0.0, 0.1, 0.2]

# make a dictionary of the grid search parameters
param_grid = dict(learn_rate=learn_rate, dropout_rate=dropout_rate)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(Final_X, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] learn_rate=0.001, dropout_rate=0.0 ..............................
[CV]  learn_rate=0.001, dropout_rate=0.0, score=0.725190853345, total=  28.1s
[CV] learn_rate=0.001, dropout_rate=0.0 ..............................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.2s remaining:    0.0s


[CV]  learn_rate=0.001, dropout_rate=0.0, score=0.763358779991, total=  27.6s
[CV] learn_rate=0.001, dropout_rate=0.0 ..............................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   55.9s remaining:    0.0s


[CV]  learn_rate=0.001, dropout_rate=0.0, score=0.830769236271, total=  27.4s
[CV] learn_rate=0.01, dropout_rate=0.0 ...............................


...

[CV]  learn_rate=0.1, dropout_rate=0.0, score=0.799999998166, total=  30.6s
[CV] learn_rate=0.001, dropout_rate=0.1 ..............................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  4.4min remaining:    0.0s


[CV]  learn_rate=0.001, dropout_rate=0.1, score=0.717557264648, total=  58.4s
[CV] learn_rate=0.001, dropout_rate=0.1 ..............................
[CV]  learn_rate=0.001, dropout_rate=0.1, score=0.763358780901, total=  57.3s
[CV] learn_rate=0.001, dropout_rate=0.1 ..............................
[CV]  learn_rate=0.001, dropout_rate=0.1, score=0.846153846154, total=  59.3s
[CV] learn_rate=0.01, dropout_rate=0.1 ...............................
[CV]  learn_rate=0.01, dropout_rate=0.1, score=0.69465648582, total=  59.1s
[CV] learn_rate=0.01, dropout_rate=0.1 ...............................
[CV]  learn_rate=0.01, dropout_rate=0.1, score=0.755725190385, total=  59.4s
[CV] learn_rate=0.01, dropout_rate=0.1 ...............................
[CV]  learn_rate=0.01, dropout_rate=0.1, score=0.807692303107, total=  59.6s
[CV] learn_rate=0.1, dropout_rate=0.1 ................................
[CV]  learn_rate=0.1, dropout_rate=0.1, score=0.755725190385, total= 1.0min
[CV] learn_rate=0.1, dropout_rate=

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed: 21.9min finished


Best: 0.783163264698, using {'learn_rate': 0.1, 'dropout_rate': 0.0}
0.772959190516 (0.0436080587255) with: {'learn_rate': 0.001, 'dropout_rate': 0.0}
0.762755097479 (0.0157114183151) with: {'learn_rate': 0.01, 'dropout_rate': 0.0}
0.783163264698 (0.0151091451193) with: {'learn_rate': 0.1, 'dropout_rate': 0.0}
0.775510209099 (0.0531670542409) with: {'learn_rate': 0.001, 'dropout_rate': 0.1}
0.752551017823 (0.046171696365) with: {'learn_rate': 0.01, 'dropout_rate': 0.1}
0.757653065178 (0.00273688574155) with: {'learn_rate': 0.1, 'dropout_rate': 0.1}
0.770408167067 (0.0280181695434) with: {'learn_rate': 0.001, 'dropout_rate': 0.2}
0.760204084218 (0.0431433153375) with: {'learn_rate': 0.01, 'dropout_rate': 0.2}
0.729591836583 (0.0253458530355) with: {'learn_rate': 0.1, 'dropout_rate': 0.2}


In [64]:
# Do a grid search to optimize kernel initialization and activation functions
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(activation, init):
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(4, input_dim = 8, kernel_initializer= init, activation= activation))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
activation = ['softmax', 'relu', 'tanh', 'linear']
init = ['uniform', 'normal', 'zero']

# make a dictionary of the grid search parameters
param_grid = dict(activation = activation, init = init)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), verbose = 10)
grid_results = grid.fit(Final_X, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................
[CV]  activation=softmax, init=uniform, score=0.725190844245, total=  18.8s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   18.9s remaining:    0.0s


[CV]  activation=softmax, init=uniform, score=0.755725200395, total=  17.8s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   36.7s remaining:    0.0s


[CV]  activation=softmax, init=uniform, score=0.82307693133, total=  17.4s
[CV] activation=softmax, init=normal .................................


...

[CV]  activation=softmax, init=zero, score=0.699999988079, total= 2.4min
[CV] activation=relu, init=uniform ...................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  4.8min remaining:    0.0s


[CV]  activation=relu, init=uniform, score=0.717557255548, total=  36.4s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.755725200395, total=  26.2s
[CV] activation=relu, init=uniform ...................................
[CV]  activation=relu, init=uniform, score=0.85384615568, total=  26.2s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.732824443861, total=  26.7s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.770992378697, total=  27.0s
[CV] activation=relu, init=normal ....................................
[CV]  activation=relu, init=normal, score=0.807692307692, total=  27.2s
[CV] activation=relu, init=zero ......................................
[CV] . activation=relu, init=zero, score=0.610687024266, total=15.3min
[CV] activation=relu, init=zero .....................................

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed: 30.8min finished


Best: 0.790816335198, using {'activation': 'linear', 'init': 'uniform'}
0.767857150308 (0.0408504761859) with: {'activation': 'softmax', 'init': 'uniform'}
0.7295918416 (0.088607730181) with: {'activation': 'softmax', 'init': 'normal'}
0.668367345874 (0.0409223170112) with: {'activation': 'softmax', 'init': 'zero'}
0.775510209099 (0.0573433211683) with: {'activation': 'relu', 'init': 'uniform'}
0.770408172845 (0.0305477581912) with: {'activation': 'relu', 'init': 'normal'}
0.668367345874 (0.0409223170112) with: {'activation': 'relu', 'init': 'zero'}
0.78826531114 (0.0301020469467) with: {'activation': 'tanh', 'init': 'uniform'}
0.790816332157 (0.0288265734517) with: {'activation': 'tanh', 'init': 'normal'}
0.668367345874 (0.0409223170112) with: {'activation': 'tanh', 'init': 'zero'}
0.790816335198 (0.0389799002513) with: {'activation': 'linear', 'init': 'uniform'}
0.78826531114 (0.0301020456743) with: {'activation': 'linear', 'init': 'normal'}
0.668367345874 (0.0409223170112) with: {'a

In [65]:
# Do a grid search to find the optimal number of neurons in each hidden layer
# import necessary packages

# Define a random seed
seed = 6
np.random.seed(seed)

# Start defining the model
def create_model(neuron1, neuron2):
    # create model
    model = Sequential()
    model.add(Dense(neuron1, input_dim = 8, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(neuron2, input_dim = neuron1, kernel_initializer= 'uniform', activation= 'linear'))
    model.add(Dense(1, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy', optimizer = adam, metrics = ['accuracy'])
    return model

# create the model
model = KerasClassifier(build_fn = create_model, epochs = 100, batch_size = 20, verbose = 0)

# define the grid search parameters
neuron1 = [4, 8, 16]
neuron2 = [2, 4, 8]

# make a dictionary of the grid search parameters
param_grid = dict(neuron1 = neuron1, neuron2 = neuron2)

# build and fit the GridSearchCV
grid = GridSearchCV(estimator = model, param_grid = param_grid, cv = KFold(random_state=seed), refit = True, verbose = 10)
grid_results = grid.fit(Final_X, Y)

# summarize the results
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))
means = grid_results.cv_results_['mean_test_score']
stds = grid_results.cv_results_['std_test_score']
params = grid_results.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print('{0} ({1}) with: {2}'.format(mean, stdev, param))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................
[CV] ....... neuron1=4, neuron2=2, score=0.770992364592, total=  13.2s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   13.2s remaining:    0.0s


[CV] ....... neuron1=4, neuron2=2, score=0.763358790001, total=  13.3s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   26.6s remaining:    0.0s


[CV] ....... neuron1=4, neuron2=2, score=0.830769236271, total=  12.9s
[CV] neuron1=4, neuron2=4 ............................................


...

[CV] ....... neuron1=4, neuron2=8, score=0.830769236271, total=  17.4s
[CV] neuron1=8, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  2.3min remaining:    0.0s


[CV] ....... neuron1=8, neuron2=2, score=0.770992364592, total=  17.6s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ....... neuron1=8, neuron2=2, score=0.770992379607, total=  17.9s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ....... neuron1=8, neuron2=2, score=0.838461545797, total=  17.8s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ........ neuron1=8, neuron2=4, score=0.75572519448, total=  19.5s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ....... neuron1=8, neuron2=4, score=0.763358790001, total=  19.8s
[CV] neuron1=8, neuron2=4 ............................................
[CV] ....... neuron1=8, neuron2=4, score=0.830769236271, total=  19.9s
[CV] neuron1=8, neuron2=8 ............................................
[CV] ....... neuron1=8, neuron2=8, score=0.770992364592, total=  23.3s
[CV] neuron1=8, neuron2=8 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  9.9min finished


Best: 0.795918374189, using {'neuron1': 16, 'neuron2': 2}
0.78826531114 (0.0301020456743) with: {'neuron1': 4, 'neuron2': 2}
0.785714288603 (0.0157927169346) with: {'neuron1': 4, 'neuron2': 4}
0.790816329116 (0.028142936874) with: {'neuron1': 4, 'neuron2': 8}
0.793367353173 (0.0317644723943) with: {'neuron1': 8, 'neuron2': 2}
0.783163272148 (0.033678636295) with: {'neuron1': 8, 'neuron2': 4}
0.78826531114 (0.0301020456743) with: {'neuron1': 8, 'neuron2': 8}
0.795918374189 (0.0353860100567) with: {'neuron1': 16, 'neuron2': 2}
0.790816330636 (0.033706161996) with: {'neuron1': 16, 'neuron2': 4}
0.78826531114 (0.0301020456743) with: {'neuron1': 16, 'neuron2': 8}


In [68]:
# generate predictions with optimal hyperparameters
y_pred = grid.predict(Final_X)

In [69]:
print(y_pred.shape)

(392, 1)


In [70]:
# Generate a classification report
from sklearn.metrics import classification_report, accuracy_score

print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

0.7806122448979592
             precision    recall  f1-score   support

          0       0.81      0.89      0.84       262
          1       0.71      0.57      0.63       130

avg / total       0.77      0.78      0.77       392

